In [ ]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

import pandas as pd
import common_utils as utils
import grouping
reload(grouping)
from grouping import Grouping
import variant_criteria
reload(variant_criteria)
from variant_criteria import Variant
import json

product_data = []

for df_chunk in pd.read_csv('/Users/a0m02fp/pcf_dump_shoes.tsv', sep='\\t', chunksize=10**4, engine='python', encoding='utf-8'):
    jsons = list(df_chunk[u'product_json'])
    
    for item in jsons:
        item = json.loads(item)
        
        if 'abstract_product_id' in item and item['abstract_product_id'] != '':
            abstract_product_id = item['abstract_product_id']
            attributes = item['product_attributes']

            attr_val_pairs = dict()

            for attr_key, vals in attributes.items():
                value = vals['values'][0]['value']
                attr_val_pairs[attr_key] = value

            item_id = attributes['item_id']['values'][0]['value'] if 'item_id' in attributes else None
            title = attributes['product_name']['values'][0]['value'] if 'product_name' in attributes else None
            long_desc = attributes['product_long_description']['values'][0]['value'] if 'product_long_description' in attributes else None
            shrt_desc = attributes['product_short_description']['values'][0]['value'] if 'product_short_description' in attributes else None
            pt = attributes['product_type']['values'][0]['value'] if 'product_type' in attributes else None
            
            variant_criterias = dict()
            
            if 'grouping' in item and 'variant' in item['grouping'] and 'attributes' in item['grouping']['variant']:
                variants = item['grouping']['variant']['attributes']
                
                for attr in variants:
                    if 'values' in variants[attr]:
                        var_attr = variants[attr]['values'][0]['value']
                        variant_criterias[attr] = var_attr
        
            product_data.append((item_id, pt, title, long_desc, shrt_desc, json.dumps(attr_val_pairs), abstract_product_id, json.dumps(variant_criterias)))

In [ ]:
df = pd.DataFrame(product_data, columns=['item_id', 'product_type', 'title', 'long_desc', 'short_desc', 'attr_val_pairs', 'abstract_product_id', 'variant_criterias'])

In [ ]:
df.to_csv('items_data_shoes.csv', sep=',', encoding='utf-8', index=None)

In [ ]:
pts = [x[1] for x in product_data]

In [ ]:
pts[0]

In [ ]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

import pandas as pd
import common_utils as utils
reload(utils)
import grouping
reload(grouping)
from grouping import Grouping
import variant_criteria
reload(variant_criteria)
from variant_criteria import Variant
import json

import siamese_network
reload(siamese_network)

import siamese_api
reload(siamese_api)

import word2vec_api
reload(word2vec_api)

In [ ]:
print "Reading data file..."

df = pd.read_csv('items_data.csv', sep=",", encoding='utf-8')
df['attr_val_pairs'] = df['attr_val_pairs'].apply(lambda x: json.loads(x))
df['variant_criterias'] = df['variant_criterias'].apply(lambda x: json.loads(x))

print "Reading product data..."
product_data = list(df.itertuples(index=False))

In [ ]:
item_type, group_type, grouping_algo = "Bras", "auto", "hierarchical"

print "Getting items..."
items = utils.get_unique_items_pt(product_data, item_type)

print len(items)

In [ ]:
print "Creating groups..."
grp_instance = Grouping(items, similarity_threshold=0.5)
grp_instance.init_groups()

print grp_instance.get_clustering_scores()

groups = grp_instance.auto_groups if group_type == 'auto' else grp_instance.true_groups
    
print "Reading excluded attribute list..."
with open('excluded_attr_list.txt', 'rb') as x_attr:
    excluded_attrs = x_attr.readlines()

excluded_attrs = [x.strip() for x in excluded_attrs] 
excluded_attrs = set(excluded_attrs)

In [ ]:
len(groups)

In [ ]:
from siamese_api import SiameseAPI
sapi = SiameseAPI(items, grp_instance.true_groups, max_num_tokens=50)
sapi.train_model()
sapi.insert_embeddings()

In [ ]:
import time
start = time.time()
d = sapi.get_nearest_neighbors_mongo(items[2][2], groups, head_thres=0.5, ind_thres=0.999)
print time.time()-start

In [ ]:
items[2][2]

In [ ]:
for x in d:
    print x

In [ ]:
s_embeds = sapi.get_representations([str(x[2]) for x in items])

In [ ]:
grp_instance1 = Grouping(items, representations=s_embeds, similarity_threshold=1.0)
grp_instance1.init_groups()
groups1 = grp_instance1.auto_groups
print grp_instance1.get_clustering_scores()

In [ ]:
len(groups1)

In [ ]:
print "Fetching important attributes..."
var_instance1 = Variant(groups1, items, max_attrs_per_var=1, max_variants=10, excluded_attrs=excluded_attrs)
valid_attrs = set([attr[0] for attr, score in var_instance1.get_variant_scores()])
    
print "Fetching variant criteria..."
var_instance = Variant(groups1, items, max_attrs_per_var=2, max_variants=1, excluded_attrs=excluded_attrs, valid_attrs=valid_attrs)
variant_scores = var_instance.get_variant_scores()

print "Predominant Variant Criteria : ", variant_scores[0]

print "Fetching variant criteria per group..."
pred_grp_variants = var_instance.get_predicted_variants()

print "Results"
var_instance.results(pred_grp_variants, variant_scores[0][0])

In [ ]:
from word2vec_api import Word2VecAPI
wapi = Word2VecAPI(items)
wapi.train_model()
wapi.insert_embeddings()

In [ ]:
import time
start = time.time()
d = wapi.get_nearest_neighbors_mongo(items[0][2], groups, head_thres=0.5, ind_thres=0.999)
print time.time()-start

In [ ]:
items[0][2]

In [ ]:
for x in d:
    print x

In [ ]:
w_embeds = wapi.get_representations([str(x[2]) for x in items])

In [ ]:
grp_instance2 = Grouping(items, representations=w_embeds, similarity_threshold=0.1)
grp_instance2.init_groups()
groups2 = grp_instance2.auto_groups
print grp_instance2.get_clustering_scores()

In [ ]:
len(groups2)

In [ ]:
print "Fetching important attributes..."
var_instance1 = Variant(groups2, items, max_attrs_per_var=1, max_variants=10, excluded_attrs=excluded_attrs)
valid_attrs = set([attr[0] for attr, score in var_instance1.get_variant_scores()])
    
print "Fetching variant criteria..."
var_instance = Variant(groups2, items, max_attrs_per_var=2, max_variants=1, excluded_attrs=excluded_attrs, valid_attrs=valid_attrs)
variant_scores = var_instance.get_variant_scores()

print "Predominant Variant Criteria : ", variant_scores[0]

print "Fetching variant criteria per group..."
pred_grp_variants = var_instance.get_predicted_variants()

print "Results"
var_instance.results(pred_grp_variants, variant_scores[0][0])

In [ ]:
from sklearn.neighbors import KDTree
import numpy as np
tree1 = KDTree(s_embeds, leaf_size=50)
tree2 = KDTree(w_embeds, leaf_size=50)

In [ ]:
ind = tree1.query_radius(s_embeds[2], r=0.02)

In [ ]:
ind = tree2.query_radius(w_embeds[2], r=0.02)

In [ ]:
for x in ind[0]:
    print items[x][2]

In [ ]:
from sklearn.neighbors import KDTree
import numpy as np
tree = KDTree(np.tile(embeds.T, 50).T, leaf_size=50)

In [ ]:
import pickle
with open('tree.pkl', 'wb') as f:
    pickle.dump(tree, f)

In [ ]:
import time, random

out = []
for j in range(5, 50, 5):
    tree = KDTree(np.tile(embeds.T, j).T, leaf_size=50)
    w = random.sample(range(len(items)), 1000)
    times = []
    for i in w:
        start = time.time()
        ind = tree.query_radius(embeds[i], r=0.02)
        duration = time.time()-start
        times.append(duration)
    out.append((j, np.max(times)))

In [ ]:
out = out + [(50, 0.028570890426635742)]

In [ ]:
import random
w = random.sample(range(len(items)), 1000)
times = []
for i in w:
    start = time.time()
    ind = tree.query_radius(embeds[i], r=0.02)
    duration = time.time()-start
    times.append(duration)

In [ ]:
175*100

In [ ]:
%matplotlib inline
from matplotlib.pyplot import plot

x, y = zip(*out)
plot(x, y)